In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d( x, W, strides = [1, strides, strides, 1], padding = 'SAME' )
    x = tf.nn.bias_add( x, b )
    return tf.nn.relu( x )

# Define model
# The idea is to keep the number of parameters similar to that of SpotNet. For that,
## Total number of 2D convolutions should match that of SpotNet. SpotNet has 30 convolutions in its input half-layer,
## and 60 convolutions in each hidden layer (of which there are 3). This amounts to a total of 210 2D convolutions
## with 4x4 kernels. 

## A convolutional layer with N input channels and M output channels implies N x M 2D convolutions
## 6+6*15+15*2+30+30+30 = 216

## Total number of bias terms and other scalar parameter should be similar to that of SpotNet. Maybe
## the number of non-linearities too? This amounts to 7 scalar parameters. The simplest structure then is 
## 7 layers with progressive growing to 30

def conv_net( image_height = 512, image_width = 512 ):
    
    # Seed for random initializations
    tf.set_random_seed(8888)
    
    # Input placeholder
    source = tf.placeholder( tf.float32, [ 1, image_height, image_width ] )
    
    # First convolutional layer, from source to six channels
    h1 = tf.Variable( tf.truncated_normal( [5, 5, 1, 6], stddev = 0.01 ) )
    x1 = tf.nn.conv2d( tf.expand_dims( source, axis = 3 ), h1, strides = [1, 1, 1, 1], padding = 'SAME' )
    b1 = tf.Variable( 1.0 )
    x1 = tf.nn.relu(  x1 + b1 )
    
    # Second convolutional layer, from six to 15 channels
    h2 = tf.Variable( tf.truncated_normal( [5, 5, 6, 15], stddev = 0.01 ) )
    x2 = tf.nn.conv2d( x1, h2, strides = [1, 1, 1, 1], padding = 'SAME' )
    b2 = tf.Variable( 1.0 )
    x2 = tf.nn.relu( x2 + b2 )
    
    # Third convolutional layer, each of the 15 channels splits in two
    h3 = tf.Variable( tf.truncated_normal( [5, 5, 15, 2], stddev = 0.01 ) )
    x3 = tf.nn.depthwise_conv2d( x2, h3, strides = [1, 1, 1, 1], padding = 'SAME' )
    b3 = tf.Variable( 1.0 )
    x3 = tf.nn.relu( x3 + b3  )
    
    x = [x3]; b = []; h = [];
    # Three equal layers in which each channel is treated independently, i.e. from 30 channels to 30 channels
    for layer_number in range(3):
        h_next = tf.Variable( tf.truncated_normal( [5, 5, 30, 1], stddev = 0.01 ) )
        x_next = tf.nn.depthwise_conv2d( x[-1], h_next, strides = [1, 1, 1, 1], padding = 'SAME' )
        b_next = tf.Variable( 1.0 )
        x_next = tf.nn.relu( x_next + b_next )
        x.append( x_next )
        b.append( b_next )
        h.append( h_next )    
    
    output = x[-1]
    
    target = tf.placeholder( tf.float32, [ 1, image_height, image_width, 30 ] )
    
    loss = tf.reduce_mean( ( output - target ) ** 2 )
    
    return (source, loss, target, output)

# Create and return the optimizer
def network_training( loss, learning_rate ):
    return tf.train.AdamOptimizer( learning_rate ).minimize( loss )

In [3]:
learning_rate = 1e-3
nrof_train_steps = 400000
nrof_cells = 1250

results_dir = 'spotnet_results/'
prefix = 'CONV-NET_%d_CELLS_%0.4f_LR_%d_STEPS_%d_BATCHSIZE'%( nrof_cells,
                                                              learning_rate,
                                                              nrof_train_steps,
                                                              1 )

# Load dataset
data = np.load( '../../sim_data/result_' + str(nrof_cells) + '_cells_10_images.npy' )[()]

nrof_cells = data['nrof_cells']

# Extract images and PSDRs
images = data['fluorospot']
psdrs = data['psdrs']

# Extract shape parameters
nrof_images, image_height, image_width = images.shape
_, _, _, nrof_kernels = psdrs.shape

# Split dataset for training and testing
nrof_training_samples = int( 0.7 * nrof_images )
train_images, train_psdrs = (images[ : nrof_training_samples, ... ], psdrs[ : nrof_training_samples, ... ])
test_images, test_psdrs = (images[nrof_training_samples : , ... ], psdrs[nrof_training_samples : , ... ])

print( 'Dataset: Num images: %d, Image height: %d, Image width: %d, Num cells: %d, Num kernels: %d'%( nrof_images, 
                                                                                                      image_height, 
                                                                                                      image_width,
                                                                                                      nrof_cells,
                                                                                                      nrof_kernels ) )

Dataset: Num images: 10, Image height: 512, Image width: 512, Num cells: 1250, Num kernels: 30


In [4]:
with tf.name_scope( 'CSCNet' ) as scope:
    source, loss, target, output = conv_net( image_height, image_width )

# Build training computational graph
with tf.name_scope( 'Training' ) as scope:
    train_step = network_training( loss, learning_rate )

In [5]:
nrof_gpu = 1 # {0, 1}
batch_size = 1
config = tf.ConfigProto( device_count = {'GPU': nrof_gpu} )

sess = tf.Session( config = config )
sess.run( tf.global_variables_initializer( ) )

In [6]:
import time

print( 'Training the network' )

# Space to store loss values
train_loss_record = np.empty( (0, ) )
test_loss_record  = np.empty( (0, ) )
iterations_record = np.empty( (0, ) )

# Indices that will be picked at random at each iteration
indices = np.arange( train_images.shape[0] )
np.random.seed(8888)

# Train the network
start = time.time()
for training_iteration in range( nrof_train_steps + 1 ):
    
    
    np.random.shuffle( indices )
    batch_indices = np.take( indices, np.arange( batch_size ), mode = 'wrap' )
    batch_input = train_images[ batch_indices, ... ]
    batch_target = train_psdrs[ batch_indices, ... ]
    
    _, train_loss = sess.run( [ train_step, loss ], feed_dict = { source: batch_input, 
                                                                  target: batch_target } )
    
    
    if training_iteration % (nrof_train_steps / 100) == 0:
        test_loss = 0
        for test_image_index in range( test_images.shape[0] ):
            test_loss += sess.run( loss, feed_dict = { source: np.expand_dims( test_images[ test_image_index, ... ], axis = 0 ), 
                                                       target: np.expand_dims( test_psdrs[  test_image_index, ... ], axis = 0 ) } )
        test_loss = test_loss / test_images.shape[0]
        test_loss_record = np.append( test_loss_record, test_loss )
        train_loss_record = np.append( train_loss_record, train_loss )
        iterations_record = np.append( iterations_record, training_iteration )
        
        if (test_loss == test_loss_record.min() and training_iteration > 10):
            print( 'Train step %d, Batch loss: %0.4f, Test loss: %0.4f, Elapsed: %ds. Best in test yet! Storing.'%( 
                                                                                    training_iteration, 
                                                                                    train_loss, 
                                                                                    test_loss, 
                                                                                    time.time() - start ) ) 
            tf.saved_model.simple_save( sess,
                results_dir + 'trained_convnet_' + str( training_iteration ) + '_train-steps_5_kersize/',
                inputs = {'image': source},
                outputs = {'psdr': output} )
        else:
            print( 'Train step %d, Batch loss: %0.4f, Test loss: %0.4f, Elapsed: %ds.'%( 
                                                                                    training_iteration, 
                                                                                    train_loss, 
                                                                                    test_loss, 
                                                                                    time.time() - start ) )
    


Training the network
Train step 0, Batch loss: 2321.3057, Test loss: 1590.5524, Elapsed: 3s.
Train step 4000, Batch loss: 1452.4852, Test loss: 1426.8415, Elapsed: 1964s. Best in test yet! Storing.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_4000_train-steps_5_kersize/saved_model.pb
Train step 8000, Batch loss: 1881.9786, Test loss: 1394.3916, Elapsed: 3909s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_8000_train-steps_5_kersize/saved_model.pb
Train step 12000, Batch loss: 1413.0874, Test loss: 1388.3988, Elapsed: 5853s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/

INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_140000_train-steps_5_kersize/saved_model.pb
Train step 144000, Batch loss: 1390.3242, Test loss: 1002.9410, Elapsed: 69421s.
Train step 148000, Batch loss: 1041.9705, Test loss: 992.8294, Elapsed: 71390s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_148000_train-steps_5_kersize/saved_model.pb
Train step 152000, Batch loss: 1095.2675, Test loss: 985.7089, Elapsed: 73346s. Best in test yet! Storing.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_152000_train-steps_5_kersize/saved_model.pb
Train step 156000, Batch loss: 1004.6012, Test loss: 994.3376, Elapsed: 75319s.
Train step 160000, Batch loss: 1284.1036, Test loss: 990.7282, Elapsed: 77294s.
Train step 164

In [7]:
tf.saved_model.simple_save(
    sess,
    results_dir + 'trained_convnet_' + str( nrof_train_steps) + '_train-steps/',
    inputs = {'image': source},
    outputs = {'psdr': output}
)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: spotnet_results/trained_convnet_400000_train-steps/saved_model.pb
